<a href="https://colab.research.google.com/github/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit/blob/master/Week%2003%20Generative%20Modeling%20II/winery-multivariate/winery-classification-gaussian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Winery classification with the multivariate Gaussian

In this notebook, we return to winery classification, using the full set of 13 features.

# Clone remote

In [1]:
import os

URL = "https://github.com/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit"
NBDIR = "Week 03 Generative Modeling II/winery-multivariate"

def clone(url, dest=".", branch="master"):
  """
  Clone remote branch from url into dest
  branch not provided: clone all branches
  """

  url = url.strip("/")
  repo = os.path.join(dest, os.path.basename(url))

  # Raise error if dest inside existing repository
  is_out = !git -C "$dest" rev-parse
  if not is_out: # inside repository
    raise ValueError("Can't clone into existing repository")
  
  # Clone specific branch
  if branch:
    !git clone --single-branch --branch "$branch" "$url" "$repo"
  # Clone all branches
  else:
    !git clone "$url" "$repo"
  os.chdir(repo)
  
  bname = !git rev-parse --abbrev-ref HEAD
  print("Current")
  print("{branch}\t{directory}".format(branch=bname, directory=os.getcwd()))

clone(URL)
%run .Importable.ipynb
%cd "$NBDIR"

Cloning into './UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 390 (delta 75), reused 6 (delta 2), pack-reused 247
Receiving objects: 100% (390/390), 2.74 MiB | 2.54 MiB/s, done.
Resolving deltas: 100% (185/185), done.
Current
['master']	/content/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit


/content/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit/Week 03 Generative Modeling II/winery-multivariate


## 1. Load in the data 

As usual, we start by loading in the Wine data set. Make sure the file `wine.data.txt` is in the same directory as this notebook.

Recall that there are 178 data points, each with 13 features and a label (1,2,3). As before, we will divide this into a training set of 130 points and a test set of 48 points.

In [0]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal, describe

In [0]:
# Load data set.
data = np.loadtxt('wine.data.txt', delimiter=',')
# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

## 2. Fit a Gaussian generative model

We now define a function that fits a Gaussian generative model to the data.
For each class (`j=1,2,3`), we have:
* `pi[j]`: the class weight
* `mu[j,:]`: the mean, a 13-dimensional vector
* `sigma[j,:,:]`: the 13x13 covariance matrix

This means that `pi` is a 4x1 array (Python arrays are indexed starting at zero, and we aren't using `j=0`), `mu` is a 4x13 array and `sigma` is a 4x13x13 array.

In [0]:
def fit_generative_model(x,y):
    k = 3  # labels 1,2,...,k
    d = (x.shape)[1]  # number of features
    mu = np.zeros((k+1,d))
    sigma = np.zeros((k+1,d,d))
    pi = np.zeros(k+1)
    for label in range(1,k+1):
        indices = (y == label)
        mu[label] = np.mean(x[indices,:], axis=0)
        sigma[label] = np.cov(x[indices,:], rowvar=0, bias=1)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, sigma, pi

In [5]:
# Fit a Gaussian generative model to the training data
mu, sigma, pi = fit_generative_model(trainx,trainy)
print(mu.shape, sigma.shape, pi.shape)

(4, 13) (4, 13, 13) (4,)


## 3. Use the model to make predictions on the test set

<font color="magenta">**For you to do**</font>: Define a general purpose testing routine that takes as input:
* the arrays `pi`, `mu`, `sigma` defining the generative model, as above
* the test set (points `tx` and labels `ty`)
* a list of features `features` (chosen from 0-12)

It should return the number of mistakes made by the generative model on the test data, *when restricted to the specified features*. For instance, using the just three features 2 (`'Ash'`), 4 (`'Magnesium'`) and 6 (`'Flavanoids'`) results in 7 mistakes (out of 48 test points), so 

        `test_model(mu, sigma, pi, [2,4,6], testx, testy)` 

should print 7/48.

**Hint:** The way you restrict attention to a subset of features is by choosing the corresponding coordinates of the full 13-dimensional mean and the appropriate submatrix of the full 13x13 covariance matrix.

In [64]:
FEATURENAMES = np.array(featurenames)
LABELNAMES = np.array([1, 2, 3])
mu_, sigma_, pi_ = mu[1:], sigma[1:], pi[1:] # remove unused values (j=0)
print(mu_.shape, sigma_.shape, pi_.shape)

def predict(obs, mu, sigma, pi):
  """
  Predict label of observation
  """
  scores = []
  for m, s, p in zip(mu, sigma, pi):
    score = np.log(p) + multivariate_normal.logpdf(obs, mean=m, cov=s)
    scores.append(score)
  return np.argmax(scores)

# Now test the performance of a predictor based on a subset of features
def test_model(mu, sigma, pi, features, tx, ty):
    ###
    ### Your code goes here
    ###
    # Restrict attention to subset of features
    mu = mu[:, features]
    sigma = sigma[np.ix_(range(sigma.shape[0]), features, features)]
    pi = pi
    tx = tx[:, features]
    
    # Predict
    predictions = np.apply_along_axis(predict, 1, tx, mu, sigma, pi)
    predicted_names = LABELNAMES[predictions]
    
    # Finally, tally up score
    errors = np.sum(predicted_names != ty)
    print("Error using {features}:\n{errors}/{ntests}".format(features=FEATURENAMES[features], errors=errors, ntests=tx.shape[0]))
    
    return errors/tx.shape[0]

test_model(mu_, sigma_, pi_, [2, 4, 6], testx, testy)

(3, 13) (3, 13, 13) (3,)
Error using ['Ash' 'Magnesium' 'Flavanoids']:
7/48


0.14583333333333334

### <font color="magenta">Fast exercises</font>

*Note down the answers to these questions. You will need to enter them as part of this week's assignment.*

Exercise 1. How many errors are made on the test set when using the single feature 'Ash'?

In [7]:
test_model(mu_, sigma_, pi_, [2], testx, testy)

Error using ['Ash']:
29/48


0.6041666666666666

Exercise 2. How many errors when using 'Alcohol' and 'Ash'?

In [8]:
test_model(mu_, sigma_, pi_, [0,2], testx, testy)

Error using ['Alcohol' 'Ash']:
12/48


0.25

Exercise 3. How many errors when using 'Alcohol', 'Ash', and 'Flavanoids'?

In [66]:
test_model(mu_, sigma_, pi_, [0,2,6], testx, testy)

Error using ['Alcohol' 'Ash' 'Flavanoids']:
3/48


0.0625

Exercise 4. How many errors when using all 13 features?

In [65]:
test_model(mu_, sigma_, pi_, range(0,13), testx, testy)

Error using ['Alcohol' 'Malic acid' 'Ash' 'Alcalinity of ash' 'Magnesium'
 'Total phenols' 'Flavanoids' 'Nonflavanoid phenols' 'Proanthocyanins'
 'Color intensity' 'Hue' 'OD280/OD315 of diluted wines' 'Proline']:
2/48


0.041666666666666664

Exercise 5. In lecture, we got somewhat different answers to these questions. Why do you think that might be?

In [82]:
# Split data in different ways (but keep training set and test set's sizes same)
runs = 10 # ways of splitting data
errors = []
for i in range(runs):
  # Split data
  perm = np.random.permutation(178)
  trainx2 = data[perm[0:130],1:14]
  trainy2 = data[perm[0:130],0]
  testx2 = data[perm[130:178], 1:14]
  testy2 = data[perm[130:178],0]

  # Fit Gaussian model
  mu2, sigma2, pi2 = fit_generative_model(trainx2, trainy2)
  mu2_, sigma2_, pi2_ = mu2[1:], sigma2[1:], pi2[1:] # remove unused values (j=0)

  # Test model
  e = test_model(mu2_, sigma2_, pi2_, range(0,13), testx2, testy2)
  errors.append(e)
display.clear_output()
print(describe(errors))

DescribeResult(nobs=10, minmax=(0.0, 0.041666666666666664), mean=0.020833333333333336, variance=0.0002893518518518518, skewness=-6.046699619459897e-16, kurtosis=-1.333333333333333)
